## Maximal Bipartite Matching for Inexact Matches Between 2 Tables

#### Goal: Explore some interesting problems around using PC's to model fuzzy joins 

#### Some examples: inexact matches between tables, finding the highest and lowest possible aggregate after a match)

Task: Build a maximal bipartite matching algorithm

Step 1: Define a graph in terms of a dictionary

In [3]:
# import dictionary for graph 
from collections import defaultdict 
  
# function for adding edge to graph 
graph = defaultdict(list) 
def addEdge(graph,u,v): 
    graph[u].append(v) 
    
# Create a graph in terms of a dictionary
def generate_edges(graph): 
    edges = [] 
  
    # for each node in graph 
    for node in graph: 
          
        # for each neighbour node of a single node 
        for neighbour in graph[node]: 
              
            # if edge exists then append 
            edges.append((node, neighbour)) 
    return edges 

Tests:

In [4]:
# declaration of graph as dictionary 
addEdge(graph,'a','c') 
addEdge(graph,'b','c') 
addEdge(graph,'b','e') 
addEdge(graph,'c','d') 
addEdge(graph,'c','e') 
addEdge(graph,'c','a') 
addEdge(graph,'c','b') 
addEdge(graph,'e','b') 
addEdge(graph,'d','c') 
addEdge(graph,'e','c') 
  
# Driver Function call  
# to print generated graph 
print(generate_edges(graph))

[('a', 'c'), ('b', 'c'), ('b', 'e'), ('c', 'd'), ('c', 'e'), ('c', 'a'), ('c', 'b'), ('e', 'b'), ('e', 'c'), ('d', 'c')]


Step 2: Detail possible edge cases for maximal bipartite matching

Step 3: Maximal Bipartite Matching Algorithm